In [2]:
import pandas as pd
import numpy as np

import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer

In [3]:
import re, os, itertools
import pandas as pd
from tqdm import tqdm

In [26]:
import spacy
import spacy.displacy as displacy

In [27]:
nlp = spacy.load("en_core_web_lg")

In [21]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)

In [4]:
train = pd.read_csv('C:/Python/4b0fc680-1-train/train/train.csv')

In [5]:
train.head()

,file,Start Date,Start Date-Answer
0,CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...,['This agreement shall begin upon the date of ...,NaN
1,EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...,"['July 11 , 2006']",7/11/2006
2,FulucaiProductionsLtd_20131223_10-Q_EX-10.9_83...,"['November 15, 2012']",11/15/2012
3,GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10...,"['Feb 10, 2014']",2/10/2014
4,IdeanomicsInc_20160330_10-K_EX-10.26_9512211_E...,"['December 21, 2015']",12/21/2015


In [6]:
train = train.rename(columns = {'file':'File_name'})

In [7]:
#train.sort_values('File_name', axis=0, inplace=True, ignore_index=True) 

In [8]:
train.head()

,File_name,Start Date,Start Date-Answer
0,CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...,['This agreement shall begin upon the date of ...,NaN
1,EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...,"['July 11 , 2006']",7/11/2006
2,FulucaiProductionsLtd_20131223_10-Q_EX-10.9_83...,"['November 15, 2012']",11/15/2012
3,GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10...,"['Feb 10, 2014']",2/10/2014
4,IdeanomicsInc_20160330_10-K_EX-10.26_9512211_E...,"['December 21, 2015']",12/21/2015


In [9]:
train.shape

(409, 3)

In [10]:
pred = pd.read_csv('C:/Python/4b0fc680-1-train/train/prediction.csv')

In [9]:
pred.head()

,file,value
0,BNCMORTGAGEINC_05_17_1999-EX-10.4-LICENSING AN...,NaN
1,CORIOINC_07_20_2000-EX-10.5-LICENSE AND HOSTIN...,NaN
2,CHANGEPOINTCORP_03_08_2000-EX-10.6-LICENSE AND...,NaN
3,INKTOMICORP_06_08_1998-EX-10.14-SOFTWARE HOSTI...,NaN
4,"ADMA BioManufacturing, LLC - Amendment #3 to ...",NaN


In [11]:
#Path to each individual txt files converted from PDF
corpus = "C:/Python/4b0fc680-1-train/train/full_contract_txt"

### Get the file names from the corpus of text documents

In [12]:
#get the files names from the corpus
text_files = []
for (dirpath, dirnames, filenames) in os.walk(corpus):
    text_files.extend(filenames)

text_files
text_files = pd.DataFrame(data = text_files, columns = ['File_name'])
#text_files.sort_values('File_name', axis=0, inplace=True, ignore_index=True) 
text_files.head()

,File_name
0,2ThemartComInc_19990826_10-12G_EX-10.10_670028...
1,ABILITYINC_06_15_2020-EX-4.25-SERVICES AGREEME...
2,ACCELERATEDTECHNOLOGIESHOLDINGCORP_04_24_2003-...
3,ACCURAYINC_09_01_2010-EX-10.31-DISTRIBUTOR AGR...
4,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...


### Extract the full contract text from each of the text file of the corpus of contracts

In [13]:
def clean(text):
    text = text.replace("\n", " ")
    regex = "_"
    subst = " "
    text = re.sub(regex, subst, text, 0)
    regex = "-----"
    subst = " "
    text = re.sub(regex, subst, text, 0)
    regex = "\*+"
    subst = "*"
    text = re.sub(regex, subst, text, 0)
    regex = "\ \.\ "
    subst = "."
    text = re.sub(regex, subst, text, 0)
    text = text.strip()
    
    return text

In [14]:
#open and read all the legal contracts text and make a list for pre processing
  
#Folder Path
path = "C:/Python/4b0fc680-1-train/train/full_contract_txt"
  
#Change the directory
os.chdir(path)

files_text = []

for file in os.listdir():
    file_path = f"{path}\{file}"
    with open(file_path, 'r',encoding='utf8') as f:
        text = f.read()
        text = clean(text)
        files_text.append(text)

In [15]:
file_text = pd.DataFrame(files_text)
file_text.head()

,0
0,CO-BRANDING AND ADVERTISING AGREEMENT THIS CO...
1,EXHIBIT 4.25 INFORMATION IN THIS EXHIBIT IDENT...
2,EXHIBIT 10.13 JO...
3,Exhibit 10.31 PURSUANT TO 17 C.F.R. § 240.2...
4,REDACTED COPY CONFIDENTIAL TREATMENT REQUESTE...


In [16]:
file_text = file_text.rename(columns = {0:'File_text'})
file_text.head()

,File_text
0,CO-BRANDING AND ADVERTISING AGREEMENT THIS CO...
1,EXHIBIT 4.25 INFORMATION IN THIS EXHIBIT IDENT...
2,EXHIBIT 10.13 JO...
3,Exhibit 10.31 PURSUANT TO 17 C.F.R. § 240.2...
4,REDACTED COPY CONFIDENTIAL TREATMENT REQUESTE...


In [17]:
file_text['File_text'] = file_text['File_text'].str.slice(0,1000)

In [18]:
file_text.head()

,File_text
0,CO-BRANDING AND ADVERTISING AGREEMENT THIS CO...
1,EXHIBIT 4.25 INFORMATION IN THIS EXHIBIT IDENT...
2,EXHIBIT 10.13 JO...
3,Exhibit 10.31 PURSUANT TO 17 C.F.R. § 240.2...
4,REDACTED COPY CONFIDENTIAL TREATMENT REQUESTE...


### Create a dataframe with the file names and texts

In [19]:
legal_contract = pd.concat([text_files, file_text], axis = 1)

In [20]:
legal_contract.head()

,File_name,File_text
0,2ThemartComInc_19990826_10-12G_EX-10.10_670028...,CO-BRANDING AND ADVERTISING AGREEMENT THIS CO...
1,ABILITYINC_06_15_2020-EX-4.25-SERVICES AGREEME...,EXHIBIT 4.25 INFORMATION IN THIS EXHIBIT IDENT...
2,ACCELERATEDTECHNOLOGIESHOLDINGCORP_04_24_2003-...,EXHIBIT 10.13 JO...
3,ACCURAYINC_09_01_2010-EX-10.31-DISTRIBUTOR AGR...,Exhibit 10.31 PURSUANT TO 17 C.F.R. § 240.2...
4,ADAMSGOLFINC_03_21_2005-EX-10.17-ENDORSEMENT A...,REDACTED COPY CONFIDENTIAL TREATMENT REQUESTE...


In [15]:
legal_contract.shape

(510, 2)

### Make the file names similar by stripping off the extensions and merge with the train data

In [21]:
#the file name in the corpus has .txt extnsion while the training data has both .pdf and .PDF extensions
legal_contract['File_name'] = legal_contract['File_name'].str.rstrip('.txt')
train['File_name'] = train['File_name'].str.rstrip('.pdf')
train['File_name'] = train['File_name'].str.rstrip('.PDF')

In [22]:
legal_final = pd.merge(train ,legal_contract, on = 'File_name')

In [21]:
legal_final.shape

(231, 4)

In [23]:
legal_final.head()

,File_name,Start Date,Start Date-Answer,File_text
0,UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.1...,[],NaN,EXHIBIT 10.11 NETWOR...
1,GluMobileInc_20070319_S-1A_EX-10.09_436630_EX-...,"['December 16, 2004']",12/16/2004,Exhibit 10.09 * CONFIDENTIAL TREATMENT REQUES...
2,GluMobileInc_20070319_S-1A_EX-10.09_436630_EX-...,[],NaN,"AMENDMENT NO. 1 Dated as of November 11, 2005..."
3,GluMobileInc_20070319_S-1A_EX-10.09_436630_EX-...,[],NaN,"AMENDMENT NO. 2 Dated as of March 27, 2006 ..."
4,GluMobileInc_20070319_S-1A_EX-10.09_436630_EX-...,[],NaN,"AMENDMENT NO. 3 Dated as of February 19, 2007..."


In [24]:
legal_final.isnull().sum()

File_name             0
Start Date            0
Start Date-Answer    83
File_text             0
dtype: int64

In [25]:
#drop the blacnk date columns
legal_final = legal_final.dropna(subset=['Start Date-Answer'])

In [26]:
legal_final.isnull().sum()

File_name            0
Start Date           0
Start Date-Answer    0
File_text            0
dtype: int64

In [28]:
legal_final.shape

(148, 4)

In [27]:
legal_final.tail()

,File_name,Start Date,Start Date-Answer,File_text
222,"LEGACYTECHNOLOGYHOLDINGS,INC_12_09_2005-EX-10....","['Dec. 8, 2005']",12/8/2005,EXHIBIT 10.2 DIS...
225,NEOMEDIATECHNOLOGIESINC_12_15_2005-EX-16.1-DIS...,['1st day of December'],12/01/[],Exhibit 16.1 A...
228,"VISIUMTECHNOLOGIES,INC_10_20_2004-EX-10.20-DIS...","['7th day of April, 2003']",4/7/2003,DISTRIBUTOR AGREEMENT This Agreement is made ...
229,ENERGYXXILTD_05_08_2015-EX-10.13-Transportatio...,"['March 11, 2015']",3/11/2015,Exhibit 10.13 TRANSPORTATION AGREEMENT BET...
230,ENTERPRISEPRODUCTSPARTNERSLP_07_08_1998-EX-10....,"['June 1, 1998<omitted>This contract shall be ...",6/1/1998,EXHIBIT 10.3 [ENTERPRISE LOGO APPEARS HERE]...


### Text annotation preparation

#### For sequence labelling in Doccano, the input need to be in JSONL format as shown below

In [ ]:
#{"text": "EU rejects German call to boycott British lamb.", "label": [ [0, 2, "ORG"], ... ]}
#{"text": "Peter Blackburn", "label": [ [0, 15, "PERSON"] ]}
#{"text": "President Obama", "label": [ [10, 15, "PERSON"] ]}

In [28]:
#First we need to make the labels from the agreement text for DATE

In [30]:
#Go through each label and find the label in the text, ensure label is pre-processed same as text.
#If labels don't match, append to a seperate file to check.

clean_text = True
djson = list()
djson_inspect = list()
for index, row in tqdm(legal_final.iterrows()):
    labels = list()
    ids = index
    file_text = row['File_text']
    
    #AGMT_DATE  
    start_date = row['Start Date']
    for date in start_date:        
        if clean_text:
            date = clean(date)
        matches = re.finditer(re.escape(date.lower()), text.lower())
        for m in matches:
            s = m.start()
            e = m.end()
            labels.append([s, e, 'AGMT_DATE'])   

148it [00:35,  4.17it/s]


In [31]:
len(labels)

205177

In [39]:
start_date

"['June 1, 1998<omitted>This contract shall be for a term of one year commencing on the date first above written;']"

In [32]:
#Check for incongruous finds, add to inspect file
flat_list = [item for sublist in labels for item in sublist]
if 'AGMT_DATE' in flat_list:
    djson.append({'text': text, "labels": labels})
else:
    djson_inspect.append({'text': text, "labels": labels})

In [50]:
djson

[{'id': 230,
  'text': 'EXHIBIT 10.3    [ENTERPRISE LOGO APPEARS HERE]  ENTERPRISE TRANSPORTATION COMPANY    a division of Enterprise Products Company    P.O. BOX 4324.PHONE 713 / 880-6500                     Contract No.  017           HOUSTON, TX 77210                                                                                                                  Date: June 1, 1998                              TRANSPORTATION CONTRACT       THIS CONTRACT is entered into by and between ENTERPRISE TRANSPORTATION COMPANY, a division of Enterprise Products Company, ("Carrier"),  and  ENTERPRISE PRODUCTS OPERATING L.P., a Delaware limited partnership - ------------------------------------------------------------------------------       P.O. Box 4324            Houston,          TX   77210     ("Shipper"). - ------------------------------------------------------------------------------      (Address)                (City)          (State)       Shipper is engaged in business as a manufactur

In [26]:
# Export the full datasets for import to Doccano
filepath = "C:/Python/4b0fc680-1-train/train/full_contract_txt"
open(filepath, 'w').write("\n".join([json.dumps(e) for e in djson]))

filepath = "C:/Python/4b0fc680-1-train/train/full_contract_txt"
open(filepath, 'w').write("\n".join([json.dumps(e) for e in djson_inspect]))

PermissionError: [Errno 13] Permission denied: 'C:/Python/4b0fc680-1-train/train/full_contract_txt'